## Using Jupyter Notebook to display Open vSwitch OVN topology
---

### Running Jupyter in a container

Dockerfile.ovn
```docker
FROM jupyter/minimal-notebook

RUN conda install -y ipywidgets matplotlib networkx && \
    pip install py2cytoscape && \
    pip install ovsdbapp visJS2jupyter
```

Build and start the container
```bash
sudo docker build -t jupyter-ovn -f Dockerfile.ovn .
sudo docker run --rm -d -p 8888:8888 jupyter-ovn start-notebook.sh --NotebookApp.token=''
```

---

In [ ]:
%matplotlib inline
import ovsdbapp.backend.ovs_idl.connection
from ovsdbapp.schema.ovn_northbound.impl_idl import OvnNbApiIdlImpl
import networkx as nx
import matplotlib.pyplot as plt
import visJS2jupyter.visJS_module
import visJS2jupyter.visualizations
import ipywidgets as widgets

#### Determining RowView Object parameters

The easiest way to determine the available parameters per logical router or switch was to run the
commands below.

```
ovn-nbctl list Logical_Router_Port
ovn-nbctl list Logical_Switch_Port
ovn-nbctl list Logical_Router_Static_Route
```

---

#### OVN Logical Router

List all the `Logical_Router` and `Logical_Router_Port`.

---

In [ ]:
def routers(idl, graph):
    router_ports = []
    for router in idl.lr_list().execute():
        graph.add_node(router.name)
        for port in router.ports:
            router_ports.append({'router': router.name, 'port': port.name, 'networks': port.networks})
            
    return router_ports

#### OVN Logical Switch

List all the `Logical_Switch` and `Logical_Switch_Ports`.  If the port type is `router` `add_edge` from the switch to the `router`.  If type is `localnet` in this case its a Layer 2 gateway.  If the type is an empty string then its an endpoint e.g. virtual machine, container, physical node.

---

In [ ]:
def switches(idl, graph, router_ports):
    for switch in idl.ls_list().execute():
        graph.add_node(switch.name)
        for port in switch.ports:
            if port.type == "router":
                router_port = port.options['router-port']
                r = next(item for item in router_ports if item["port"] == router_port)
                graph.add_edge(switch.name, r["router"], network=str(r["networks"]))
            if port.type == "localnet":
                graph.add_node(port.name)
                graph.add_edge(switch.name, port.name, network='')
            # If type is blank this is most likely an endpoint e.g. virtual machine, container, physical node 
            if port.type == "":
                graph.add_node(port.name)
                graph.add_edge(switch.name, port.name, network='')

#### Render networkx graph using visJS2jupyter

In [ ]:
def render_graph(graph):
    graph_title = 'Open Virtual Network Topology'
    pos = nx.circular_layout(graph)
    nodes = graph.nodes()
    edges = graph.edges()
    nodes_dict = [{"id":n,
                   "degree":nx.degree(graph,n),
                  "x":pos[n][0]*1000,
                  "y":pos[n][1]*1000} for n in nodes
                  ]

    network = nx.get_edge_attributes(graph, 'network')

    node_map = dict(zip(nodes,range(len(nodes))))
    edges_dict = [{"source":node_map[x], "target":node_map[y], 
                  "id":network[(x,y)]} for x,y in edges]

    return visJS2jupyter.visJS_module.visjs_network(nodes_dict, edges_dict,
                                             graph_title=graph_title, graph_width=800, graph_height=800,
                                             border_color='black',
                                             node_size_multiplier=7,
                                             node_size_transform = '',
                                             node_color_highlight_border='red',
                                             node_color_highlight_background='#D3918B',
                                             node_color_hover_border='blue',
                                             node_color_hover_background='#8BADD3',
                                             node_font_size=25,
                                             edge_arrow_to=False,
                                             physics_enabled=True,
                                             edge_color_highlight='#8A324E',
                                             edge_color_hover='#8BADD3',
                                             edge_width=3,
                                             edge_font_align='top',
                                             edge_font_size=12,
                                             max_velocity=15,
                                             min_velocity=1)

In [ ]:
text = widgets.Text(description="northd:", value='172.30.1.10', width=200)
button = widgets.Button(description="Connect")

In [ ]:
def on_button_clicked(b):
    global graph_output
    graph = nx.Graph()
    ovsidl = ovsdbapp.backend.ovs_idl.connection.OvsdbIdl.from_server("tcp:%s:6641" % text.value, 'OVN_Northbound')
    ovsdb_connection = ovsdbapp.backend.ovs_idl.connection.Connection(idl=ovsidl,timeout=100)
    idl = OvnNbApiIdlImpl(ovsdb_connection)
    router_ports = routers(idl, graph)
    switches(idl, graph, router_ports)
    display(render_graph(graph))

container = widgets.HBox([text,button])
display(container)
button.on_click(on_button_clicked)